In [ ]:
# Install neccessary libraries
!pip install transformers datasets peft pandas scikit-learn -q

In [ ]:
# Import libraries
import torch
from datasets import load_dataset, Dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    pipeline
)
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
import peft
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Some function to take a look onto the trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
# Setup the base structure and repository
model_id = "roberta-base"
repository_id = "roberta-base-fine-tuned"
label_column = "ground_truth"

In [ ]:
# Load the data
df = pd.read_parquet('automotive_failure_mode_comments.parquet')
print(df.keys())
print(df[label_column].unique())

In [ ]:
# Prepare training and validation datasets
df = pd.read_parquet('automotive_failure_mode_comments.parquet')
df_fine_tuning = pd.DataFrame()
df_fine_tuning["text"] = df["comment"]
df_fine_tuning["label"] = df[label_column]

# Mapping from label to integer (and vice versa)
labels = df_fine_tuning["label"].unique()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
df_fine_tuning["label"] = df_fine_tuning["label"].map(label2id)

train_text, val_text, train_labels, val_labels = train_test_split(
    df_fine_tuning["text"].tolist(), df_fine_tuning["label"].tolist(), test_size=0.2, random_state=1909
)
train_data = {"text": train_text}
train_data["label"] = train_labels
train_dataset = Dataset.from_dict(train_data).with_format("torch")
val_data = {"text": val_text}
val_data["label"] = val_labels
val_dataset = Dataset.from_dict(val_data).with_format("torch")

# Initialize tokenizer to preprocess the data
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

In [ ]:
# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Setup/configure the training

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)
print_trainable_parameters(model)

# Create PEFT model for training using LoRA
config = LoraConfig(
    task_type=peft.utils.TaskType.SEQ_CLS,
    r=16,
    inference_mode=False,
    lora_alpha=16,
    lora_dropout=0.1,
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=30,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=False,
)

# Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Save our model
lora_model.save_pretrained(repository_id)